# Cookie & Session & Authentication

## Cookies

- 웹서버가 웹브라우져에 필요한 정보들을 저장해놓고 사용하는 것
- 쇼핑카트 서비스에 활용
- 참조 : documentaion of [req.cookie](http://expressjs.com/en/4x/api.html#req.cookies) 
- install cookie-parser package by `npm install cookie-parser --save`

In [ ]:
var express = require('express'):
var cookieParser = require('cookie-parser')
var app = express();

app.use(cookieParser()):
app.get('/count', function(req, res) {
    if(req.cookie.count) {
        var count = parseInt(req.cookies.count);
    } else {
        var count = 0;
    }
    count = count + 1;
    res.cookie('count', count);
    res.send('Count : ' + count);
});
app.listen(3000, function(){
    console.log('Connected 3000 port!'):
});

### shorping cart 구현

In [ ]:
var products = {
    1:{title:'The history of web 1'},
    2:{title:'The next web'}
};

app.get('/products', function(req, res){
    var output = '';
    for(var name in products) {
        output += `
            <li>
                <a href="/cart/${name}">${products[name].title}</a>
            </li>
            `
    }
    res.send(`<h1>Products</h1> <ul>${output}</ul> <a href="/cart"> Cart</a>`);
});

app.get('/cart/:id', function(req,res){
    var id = req.params.id;
    if(req.cookies.cart) {
        var cart = req.cookies.cart;
    } else {
        var cart = {};
    }
    if(!card[id]) {
        cart[id] = 0;
    }
    cart[id] = parseInt(cart[id]) + 1;
    res.cookie('cart', cart);
    res.send(cart);
};

- Try as follows `localhost:3000/cart/1` &`localhost:3000/cart/2` &`localhost:3000/cart/1`
- Now, it's time to decorate cart page

In [ ]:
app.get('/cart', function(req, res){
    var cart = req.cookies.cart;
    if(!cart) {
        var cart = req.cookies.cart;
    } else {
        var output = ``;
        for (var id in cart) {
            output += '<li> ${products[id].title} (${cart[id]}) </li>';
        }
    }
    res.send(`
        <h1>CArt</h1>
        <ul>${output}</ul>
        <a href="/products">Products List</a>
        `);
}

Exercise : 삭제 기능 추가

# Session

- 웹서버가 사용자 식별자인 id만을 웹브라우져에 저장하고 이에 해당하는 정보들을 서버에 놓고 서비스에 활용하는 방법
- 로그인 상태를 유지하며 서비스를 제공하는데 활용
- `npm install express-session --save` 로 express-session package install 

In [ ]:
var express = require('express');
var session = require('express-session');

var app = express();
app.use(session({
    secret: 'asdjfjkajdkjas2323454',
    resave: false,
    saveUninitialized: true
}));

app.get('/count', function(req, res){
    if(req.session.count) {
        req.session.count++;
    } else {
        req.session.count = 1;
    }
    res.send('count : '+req.session.cout);
});

app.listen(3000, function(){
    console.log('Connected 3000 port!');
});


### Session 을 Login 에 활용하는 법

In [ ]:
app.get('/auth/login', function(req, res){
    var output = `
    <h1>Login</h1>
    <form action="/auth/login" method="post">
        <p>
            <input type="text" name="username" placeholder="username">
        </p>
        <p>
            <input type="password" name="password" placeholder="password">
        </p>
        <p>
            <input type="submit"
        </p>
    </form>
    `;
    res.send('Hello Login');
});

app.get('/welcome', function(req, res) {
    if(req.session.displayName) {
        res.send(`
            <h1>hello, ${req.session.displayName} </h1>
            <a href="/login/logout">Logout</a>
            `);
    } else {
        res.send(`
        <h1>Welcome</h1>
        <a href="/suth/login">Login</a>
        `);
    }
});

app.get('/auth/logout', function(req, res) {
    delete req.session.displayName;
    res.redirect('/welcome'); // res.send('Hi Logout');
}
var bodyParser = require('body-parser');
app.use(bodyParser.urlencoded({extended:fase}));

spp.post('/auth/login', function(req, res){
    var uname = req.body.username;
    var pwd = req.body.password;
    // Check username in the database 
    var user = {
        username:'junji',
        password:'1111',
        displayName:'Jun Ji'
    }
    if(uname == user.username && pwd == user.password){
        req.session.displayName = user.displayName;
        res.redirect('/welcome'); // res.send('Welcome ');
    } else {
        res.send('Who are you? <a href="/suth/login"> login again </a>');
    }
}

- session 정보를 파일로 저장하여 사용하는 방법
- `npm install session-file-store --save` 로 필요한 패키지 설치
- 

In [ ]:
var FileStore = require('session-file-store')(session);

app.use(session({
    secret: 'asdjfjkajdkjas2323454',
    resave: false,
    saveUninitialized: true,
    store: new FileStore()
}));

- 실핼시 `session` 이라는 폴더가 생성.

## Security (사용자 등록 및 인증)

- `npm install md5 --save`
- try : node => var md5 = require('md5'); md5('1111') => use the encoded password as password
- using salt : var salt = 'adk$!$%$asdj'; var pwd = '1111'; md5(pwd+salt); => use this instead

In [ ]:
var md5 = require('md5');

var users = [
    {
    username:'junji',
    password:'adkjiakjfdja',
    salt:'adk$!$%$asdj'
    displayName:'Jun Ji'
    },
    {
    username:'KKK',
    password:' ',
    salt:'adjfadk',
    displayName:'Kim Kee'
    }
]

...
if(uname == user.username && md5(pwd+user.salt) == user.password) {

}
...

- `npm install sha256 --save` 

In [ ]:
var sha256 = require('sha256')

...
if(uname == user.username && sha256(pwd+user.salt) == user.password) {

}
...

- `npm install pbkdf2-password --save`
- try : 
'''
var bkfd2Password(); 
hasher({password:'1111'}, function(err, pass, salt, hash){
... console.log(err,pass,salt,hash); });
'''

In [ ]:
var user = [
    {
    }
]

if(uname === user.username) {
    return hasher({password:pwd, salt:user.salt}, function(err, pass, salt, hash){
        if(hash === user.password) {
           req.session.displayName = user.displayName;
           req.session.save(function(){
           res.redirect('/welcome');
           })
        } else {
            res.send('Who are you? <a href="/auth/login"> loign </a>'
        }    
    });
}

####  Saving password

In [ ]:
app.post('/auth/register', function(req, res){
    hasher({password:req.body.password}, function(err, pass, salt, hash) {
        var user = {
            username: req.body.username,
            password:hash,
            salt:salt,
            displayName:req.body.displayName
        };
        users.push(user);
        req.session.displayName = req.body.displayName;
        req.session.save(function(){
            res.redirect('/welcom');
        });
    });
});

## PassportJS

- 자체인증(Local authentication) 과 타사인증(Federation authentication) 구현 및 관리에 편리.
- 참조 : http://passportjs.org
- `npm install --save passport passport-local`
- Note : 사용자 
- Middleware 는 callback 함수를 반환해준다고 생각하면 됨

### Local authentication

In [ ]:
var express = require('express');
var session = require('express-session');
var FileStore = require('session-file-store')(session);
var bodyParser = require('ody-parser');
var bkfd2Password = require('pbkdf2-password');
var passport = require('passport')
var LocalStrategy = require('passport-local').Strategy;
var hasher = bkfd2Password();

var app = express();

app.use(bodyParser.urlencoded({ extended: false }));
app.use(session({
    secret: 'adlklaksdjas;dj',
    resave: false,
    saveUninitialized: true,
    store: new FileStore()
}));
app.use(passport.initialize());  // should be after app.use(session())
app.use(passport.session());

app.post('/auth/login',
         passport.authenticate('local', 
                        { successRedirect: '/welcome',
                          failureRedirect: '/auth/login',
                          failureFlash: false })
);
         
passport.use(new LocalStrategy(
    function(username, password, done) {
        User.findOne({ username: username }, function(err, user) {
            if (err) { return done(err); }
            if (!user) {
                return done(null, false, { message: 'Incorrect username.' });
            }
            if (!user.validPassword(password)) {
                return done(null, false, { message: 'Incorrect password.' });
            }
                return done(null, user);
        });
      }
));
```
passport.use(new LocalStrategy(
    function(username, password, done){
        var uname = username;
        var pwd = password;
        for(var i=0; i<users.length; i++){
            var user = users[i];
            if(uname == user.username){
                return hasher({password:pwd, salt:user.salt}, function(err,pass, salt, hash){
                    if(hash == user.password){
                        done(null, user);
                    } else {
                        done(null, false);
                    }
                });
            }
        }
        done(null, false);
    }
));
```
passport.serializeUser(function(us er, done) {
        done(null, user.id);
});

passport.deserializeUser(function(id, done) {
    User.findById(id, function(err, user) {
        done(err, user);
    });
});


passport.deserializeUser(function(id, done) {
    for(var i=0; i<users.length; i++) {
        var user = users[i];
        if(user.username == id) {
        
        }
    }
    User.findById(id, function(err, user) {
        done(err, user);
    });
});


app.get('/logout', function(req, res){
    req.logout();
    res.redirect('/welcome');
});


### 타사인증 (Federation authentication)
- 설치 : `npm install passport-facebook --save`